In [1]:
from dataloader import Dataset
from utils import * 

data = Dataset("Stage_1_Data_ver_4/stage1/box")
data.load_data()

In [2]:
camera_visibility = data.visibility_new.any(axis = 1).astype(np.float64)

In [3]:
images_to_add = data.images
appended_ids = np.arange(data.N_images)



In [4]:
from scipy.spatial.transform import Rotation 
from scipy.sparse import lil_matrix
import time
from scipy.optimize import least_squares
def computeCurrentVisibility(visibility, reconstructed_ind, num_views, appended_ids):

    X_indices = (reconstructed_ind == 1)[:,0]
    current_visibility = visibility[:,appended_ids[:num_views]][X_indices]
    
    return X_indices, current_visibility

def computeCurrent2DPoints(X_indices, points_2d, appended_ids, current_visibility):
    
    current_2d_points = points_2d[:, appended_ids[:current_visibility.shape[1]]][X_indices]
    point_indices, camera_indices = current_visibility.nonzero()
    return current_2d_points[point_indices, camera_indices], point_indices, camera_indices
    
def projectFromCameras(X, cam_params, camera_indices, point_indices, K):

    projected = np.zeros((X.shape[0], cam_params.shape[0], 3))

    for camera in range(cam_params.shape[0]):

        R = Rotation.from_rotvec(cam_params[camera,:3]).as_matrix()
        C = cam_params[camera, 3:]
        P = computeProjectionMatrix(K, R, C)
        
        projected[:, camera] = projectPoint(P, X)

    return projected[point_indices, camera_indices][:,:2]

def objective(x0, num_views, n_points, camera_indices, point_indices, points_2d, K):
    
    camera_params = x0[:num_views * 6].reshape((num_views, 6))
    points_3d = x0[num_views * 6:].reshape((n_points, 3))
    points_proj = projectFromCameras(points_3d, camera_params,camera_indices,point_indices,K)
    error_vec = (points_proj - points_2d).ravel()

    return error_vec

def bundle_adjustment_sparsity(num_views, n_points, camera_indices, point_indices):
    m = camera_indices.size * 2
    n = num_views * 6 + n_points * 3
    A = lil_matrix((m, n), dtype=int)

    i = np.arange(camera_indices.size)
    for s in range(6):
        A[2 * i, camera_indices * 6 + s] = 1
        A[2 * i + 1, camera_indices * 6 + s] = 1

    for s in range(3):
        A[2 * i, num_views * 6 + point_indices * 3 + s] = 1
        A[2 * i + 1, num_views * 6 + point_indices * 3 + s] = 1

    return A

def bundleAdjustment(X_reconstructed, reconstructed_ind, points_2d, visibility, camera_rotations, camera_translations, K, num_views, appended_ids):
    
    X_indices, current_visibility = computeCurrentVisibility(visibility, reconstructed_ind, num_views, appended_ids)
    X_3d = X_reconstructed[X_indices]
    n_points = X_3d.shape[0]
    X_2d,point_indices, camera_indices = computeCurrent2DPoints(X_indices, points_2d, appended_ids, current_visibility)

    cam_param_list = []
    for i in range(num_views):
        R, C = camera_rotations[i], camera_translations[i]
        rotvec = Rotation.from_matrix(R).as_rotvec()
        RC = [rotvec[0], rotvec[1], rotvec[2], C[0], C[1], C[2]]
        cam_param_list.append(RC)

    cam_param_list = np.array(cam_param_list).reshape(-1, 6)

    x0 = np.hstack((cam_param_list.ravel(), X_3d.ravel()))

    A = bundle_adjustment_sparsity(num_views,n_points, camera_indices, point_indices)
    res = least_squares(objective, x0, jac_sparsity=A, verbose=0, x_scale='jac', max_nfev=50, method='trf',
                        args=(num_views, n_points, camera_indices, point_indices,X_2d, K))
    
    optimized_params = res.x
    optimized_RC = optimized_params[:num_views * 6].reshape((num_views, 6))
    optimized_3d = optimized_params[num_views * 6:].reshape((n_points, 3))

    optimized_X = np.zeros_like(X_reconstructed)
    optimized_X[X_indices] = optimized_3d

    optimized_camera_rotations, optimized_camera_translation = [], []
    for i in range(len(optimized_RC)):
        R = Rotation.from_rotvec(optimized_RC[i, :3]).as_matrix()
        C = optimized_RC[i, 3:].reshape(3,1)
        optimized_camera_translation.append(C)
        optimized_camera_rotations.append(R)

    return optimized_X, optimized_camera_rotations, optimized_camera_translation


In [13]:

indices    = data.visibility_new[:,appended_ids[0], appended_ids[1]] == 1
pts1, pts2 = data.points[indices, appended_ids[0]], data.points[indices, appended_ids[1]]
E = estimateEssentialFromFundamental(estimate_fundamental_matrix(pts1, pts2), data.K)
rotations, translations = decomposeEssentialMat(E)
points_3d = []
for i in range(4):
    P1 = computeProjectionMatrix(data.K, np.identity(3), np.zeros((3,)))
    P2 = computeProjectionMatrix(data.K, rotations[i], translations[i])
    points_3d.append(linearTriangulation(pts1, pts2, P1, P2))

X,R,C = cheiralityCondition(points_3d, rotations, translations)

X_reconstructed   = np.zeros((data.points.shape[0], 3))
reconstructed_ind = np.zeros((data.points.shape[0], 1))

X_reconstructed[indices] = X
reconstructed_ind[indices] = 1

camera_rotations    = []
camera_translations = []
projection_matrix_extrinsic = []

camera_rotations.append(np.eye(3))
camera_rotations.append(R)

camera_translations.append(np.zeros((3,)))
camera_translations.append(C)

projection_matrix_extrinsic.append(projectionMatrix4x4(computeProjectionMatrix(np.identity(3), camera_rotations[0], camera_translations[0])))
projection_matrix_extrinsic.append(projectionMatrix4x4(computeProjectionMatrix(np.identity(3), camera_rotations[1], camera_translations[1])))



In [14]:

for next_image in range(2, 10):
    print(next_image)
    visibility_points = data.visibility_new[:, appended_ids[next_image - 1], appended_ids[next_image]]
    indices           = visibility_points == 1
    pts1, pts2 = data.points[:, appended_ids[next_image -1]], data.points[:, appended_ids[next_image]]
    E = estimateEssentialFromFundamental(estimate_fundamental_matrix(pts1[indices], pts2[indices]), data.K)
    rs, ts = decomposeEssentialMat(E)


    indices_new = np.logical_and((1 - reconstructed_ind)[:,0] , visibility_points)
    points_3d = []
    for i in range(4):
        P1 = computeProjectionMatrix(data.K, np.identity(3), np.zeros((3,)))
        P2 = computeProjectionMatrix(data.K, rs[i], ts[i])
        points_3d.append(linearTriangulation(pts1[indices_new], pts2[indices_new], P1, P2))

    X,R,C = cheiralityCondition(points_3d, rs, ts)

    X_reconstructed[indices_new] = X
    reconstructed_ind[indices_new] = 1
    
    relative_projection = projectionMatrix4x4(computeProjectionMatrix(np.identity(3),R,C))
    absolute_RC = np.matmul(relative_projection, projection_matrix_extrinsic[next_image -1])
    camera_rotations.append(absolute_RC[:3,:3])
    camera_translations.append(np.matmul(- np.linalg.inv(absolute_RC[:3,:3]), absolute_RC[:-1,3].reshape(3,1)))
    projection_matrix_extrinsic.append(absolute_RC)
    initial = projection_matrix_extrinsic.copy()
    for previous_images in range(next_image -1):
    
        visibility_points = data.visibility_new[:, appended_ids[previous_images], appended_ids[next_image]]
        indices_new = np.logical_and((1 - reconstructed_ind)[:,0] , visibility_points)
        pts1, pts2 = data.points[:, appended_ids[previous_images]], data.points[:, appended_ids[next_image]]
        P1 = computeProjectionMatrix(data.K, camera_rotations[previous_images], camera_translations[previous_images])
        P2 = computeProjectionMatrix(data.K, camera_rotations[next_image], camera_translations[next_image])
        points_3d = linearTriangulation(pts1[indices_new], pts2[indices_new], P1, P2)
        X_reconstructed[indices_new]   = points_3d
        reconstructed_ind[indices_new] = 1

    
    
    # X_reconstructed,camera_rotations,camera_translations = bundleAdjustment(X_reconstructed,reconstructed_ind,
    #                                                                         data.points,camera_visibility,camera_rotations,
    #                                                                         camera_translations, data.K, next_image +1, appended_ids)
    
    
    # projection_matrix_extrinsic = []
    # I_p = projectionMatrix4x4(computeProjectionMatrix(np.identity(3), camera_rotations[0], camera_translations[0]))
    
    # projection_matrix_extrinsic.append(I_p)
    # projection_matrix_extrinsic.append(projectionMatrix4x4(computeProjectionMatrix(np.identity(3), camera_rotations[1], camera_translations[1])))

    # for view in range(2, len(camera_rotations)):
    #     current = projectionMatrix4x4(computeProjectionMatrix(np.identity(3), camera_rotations[view], camera_translations[view]))
    #     projection_matrix_extrinsic.append(np.matmul(current, projection_matrix_extrinsic[view -1]))

    

2
3
4
5
6
7
8
9


In [19]:
from natsort import natsorted
correspondence_files = natsorted(glob.glob(f"/home/parikshat/MyWorkings/Masters/Semester_2/3d_vision/Project/Stage_1_Data_ver_4/stage1/box/correspondences/*"))

['/home/parikshat/MyWorkings/Masters/Semester_2/3d_vision/Project/Stage_1_Data_ver_4/stage1/box/correspondences/0_1.txt', '/home/parikshat/MyWorkings/Masters/Semester_2/3d_vision/Project/Stage_1_Data_ver_4/stage1/box/correspondences/1_2.txt', '/home/parikshat/MyWorkings/Masters/Semester_2/3d_vision/Project/Stage_1_Data_ver_4/stage1/box/correspondences/1_3.txt', '/home/parikshat/MyWorkings/Masters/Semester_2/3d_vision/Project/Stage_1_Data_ver_4/stage1/box/correspondences/2_3.txt', '/home/parikshat/MyWorkings/Masters/Semester_2/3d_vision/Project/Stage_1_Data_ver_4/stage1/box/correspondences/2_4.txt', '/home/parikshat/MyWorkings/Masters/Semester_2/3d_vision/Project/Stage_1_Data_ver_4/stage1/box/correspondences/3_4.txt', '/home/parikshat/MyWorkings/Masters/Semester_2/3d_vision/Project/Stage_1_Data_ver_4/stage1/box/correspondences/3_5.txt', '/home/parikshat/MyWorkings/Masters/Semester_2/3d_vision/Project/Stage_1_Data_ver_4/stage1/box/correspondences/4_5.txt', '/home/parikshat/MyWorkings/Mas

In [ ]:
import  struct
def write_pointcloud(filename,xyz_points,rgb_points=None):

    """ creates a .pkl file of the point clouds generated
    """

    assert xyz_points.shape[1] == 3,'Input XYZ points should be Nx3 float array'
    if rgb_points is None:
        rgb_points = np.ones(xyz_points.shape).astype(np.uint8)*255
    assert xyz_points.shape == rgb_points.shape,'Input RGB colors should be Nx3 float array and have same size as input XYZ points'

    # Write header of .ply file
    fid = open(filename,'wb')
    fid.write(bytes('ply\n', 'utf-8'))
    fid.write(bytes('format binary_little_endian 1.0\n', 'utf-8'))
    fid.write(bytes('element vertex %d\n'%xyz_points.shape[0], 'utf-8'))
    fid.write(bytes('property float x\n', 'utf-8'))
    fid.write(bytes('property float y\n', 'utf-8'))
    fid.write(bytes('property float z\n', 'utf-8'))
    fid.write(bytes('property uchar red\n', 'utf-8'))
    fid.write(bytes('property uchar green\n', 'utf-8'))
    fid.write(bytes('property uchar blue\n', 'utf-8'))
    fid.write(bytes('end_header\n', 'utf-8'))

    # Write 3D points to .ply file
    for i in range(xyz_points.shape[0]):
        fid.write(bytearray(struct.pack("fffccc",xyz_points[i,0],xyz_points[i,1],xyz_points[i,2],
                                        rgb_points[i,0].tostring(),rgb_points[i,1].tostring(),
                                        rgb_points[i,2].tostring())))
    fid.close()


In [ ]:
X = X_reconstructed[(reconstructed_ind == 1)[...,0]]
c = data.colors[(reconstructed_ind == 1)[...,0]]

In [ ]:
X_reconstructed.min()

In [ ]:
write_pointcloud("initial_run.ply", X, c)

In [ ]:
R,t = PnPRANSAC(data.K, corr_points, X_new)

In [ ]:
indices = (visibility[:,37, 40] == 1)
print(indices.sum())
pts1, pts2 = data.points[indices, 37], data.points[indices, 40]
    

In [ ]:
projection_matrix_extrinsic[:3]

In [16]:
import json 
with open("/home/parikshat/MyWorkings/Masters/Semester_2/3d_vision/Project/Stage_1_Data_ver_4/stage1/box/gt_camera_parameters.json") as json_data:
    camera_parameters = json.load(json_data)
    json_data.close()


In [17]:
np.array(camera_parameters['extrinsics']['00007.jpg'])

array([[ 0.13224099,  0.33209007, -0.93393135,  5.0117027 ],
       [-0.37734979,  0.88812644,  0.26237109, -0.79974682],
       [ 0.91657926,  0.3177235 ,  0.24276078,  5.02277276],
       [ 0.        ,  0.        ,  0.        ,  1.        ]])

In [18]:
projection_matrix_extrinsic[7]

array([[ 0.10827783,  0.29242342, -0.95013918,  3.49098547],
       [-0.38049829,  0.89517045,  0.23214418, -1.12122662],
       [ 0.91842091,  0.33639026,  0.20819372,  5.48195341],
       [ 0.        ,  0.        ,  0.        ,  1.        ]])

In [ ]:
computeProjectionMatrix(data.K,R,C)

In [ ]:
q, A = test(pts1, pts2, P1, computeProjectionMatrix(data.K,R,C))

In [ ]:
(new / new[:,3][...,None]).min()

In [ ]:
new = Lin_tri(data.K, camera_translations[0], camera_rotations[0], C,R,pts1, pts2)


In [ ]:
q == new

In [ ]:
def PM(K,R,C):

    C = np.reshape(C, (3, 1))        
    I = np.identity(3)
    P = np.dot(K, np.dot(R, np.hstack((I, -C))))

    return P


In [ ]:
computeProjectionMatrix(data.K, R,C)

In [ ]:
PM(data.K, R,C)

In [ ]:
ProjectionMatrix(R,C,data.K)

In [ ]:
import scipy.optimize as optimize
def NonLinearTriangulation(K, pts1, pts2, x3D, R1, C1, R2, C2):
    """    
    K : Camera Matrix
    pts1, pts2 : Point Correspondences
    x3D :  initial 3D point 
    R2, C2 : relative camera pose
    Returns:
        x3D : optimized 3D points
    """
    
    P1 = ProjectionMatrix(R1,C1,K) 
    P2 = ProjectionMatrix(R2,C2,K)
    # pts1, pts2, x3D = pts1, pts2, x3D
    
    if pts1.shape[0] != pts2.shape[0] != x3D.shape[0]:
        raise 'Check point dimensions - level nlt'

    x3D_ = []
    for i in range(len(x3D)):
        optimized_params = optimize.least_squares(fun=ReprojectionLoss, x0=x3D[i], method="trf", args=[pts1[i], pts2[i], P1, P2])
        X1 = optimized_params.x
        x3D_.append(X1)
        # x3D_.append(X1[:3])
    return np.array(x3D_)


def ReprojectionLoss(X, pts1, pts2, P1, P2):
    
    # X = homo(X.reshape(1,-1)).reshape(-1,1) # make X a column of homogenous vector
    
    p1_1T, p1_2T, p1_3T = P1 # rows of P1
    p1_1T, p1_2T, p1_3T = p1_1T.reshape(1,-1), p1_2T.reshape(1,-1),p1_3T.reshape(1,-1)

    p2_1T, p2_2T, p2_3T = P2 # rows of P2
    p2_1T, p2_2T, p2_3T = p2_1T.reshape(1,-1), p2_2T.reshape(1,-1), p2_3T.reshape(1,-1)

    ## reprojection error for reference camera points - j = 1
    u1,v1 = pts1[0], pts1[1]
    u1_proj = np.divide(p1_1T.dot(X) , p1_3T.dot(X))
    v1_proj =  np.divide(p1_2T.dot(X) , p1_3T.dot(X))
    E1= np.square(v1 - v1_proj) + np.square(u1 - u1_proj)

    
    ## reprojection error for second camera points - j = 2    
    u2,v2 = pts2[0], pts2[1]
    u2_proj = np.divide(p2_1T.dot(X) , p2_3T.dot(X))
    v2_proj =  np.divide(p2_2T.dot(X) , p2_3T.dot(X))    
    E2= np.square(v2 - v2_proj) + np.square(u2 - u2_proj)
    
    error = E1 + E2
    return error.squeeze()

In [ ]:
new = new / new[:,3][...,None]

In [ ]:
X = NonLinearTriangulation(data.K,pts1, pts2, new,camera_rotations[0], camera_translations[0], R,C )

In [ ]:
(X/X[:,3].reshape(-1,1)).min()

In [ ]:
import scipy.optimize as optimize
import numpy as np

def NonLinearPnP(K, pts, x3D, R0, C0):
    """    
    K : Camera Matrix
    pts1, pts2 : Point Correspondences
    x3D :  initial 3D point 
    R2, C2 : relative camera pose - estimated from PnP
    Returns:
        x3D : optimized 3D points
    """

    Q = Rotation.from_matrix(R0).as_quat()
    X0 = [Q[0] ,Q[1],Q[2],Q[3], C0[0], C0[1], C0[2]] 

    optimized_params = optimize.least_squares(
        fun = PnPLoss,
        x0=X0,
        method="trf",
        args=[x3D, pts, K])
    X1 = optimized_params.x
    Q = X1[:4]
    C = X1[4:]
    R = Rotation.from_quat(Q).as_matrix()
    return R, C

def PnPLoss(X0, x3D, pts, K):
    
    Q, C = X0[:4], X0[4:].reshape(-1,1)
    R = Rotation.from_quat(Q).as_matrix()
    P = computeProjectionMatrix(K,R,C)
    
    Error = []
    for X, pt in zip(x3D, pts):

        p_1T, p_2T, p_3T = P# rows of P
        p_1T, p_2T, p_3T = p_1T.reshape(1,-1), p_2T.reshape(1,-1), p_3T.reshape(1,-1)


        X = to_homogeneous(X.reshape(1,-1)).reshape(-1,1) # make X it a column of homogenous vector
        ## reprojection error for reference camera points 
        u, v = pt[0], pt[1]
        u_proj = np.divide(p_1T.dot(X) , p_3T.dot(X))
        v_proj =  np.divide(p_2T.dot(X) , p_3T.dot(X))

        E = np.square(v - v_proj) + np.square(u - u_proj)

        Error.append(E)

    sumError = np.mean(np.array(Error).squeeze())
    return sumError

In [ ]:
R,C = DLT_PnP(corr_points, X_new, data.K)

In [ ]:
R

In [ ]:
C

In [ ]:
Rn,Cn = NonLinearPnP(data.K, corr_points, X_new, R,C)

In [ ]:
Rn

In [ ]:
Cn

In [ ]:
R

In [ ]:
C

In [ ]:
X = linearTriangulation(pts1, pts2)

In [ ]:
pts1.shape

In [ ]:
pts2.shape

In [ ]:
pts1

In [ ]:
appended_ids[:3]

In [ ]:
images_to_add[:3]

In [ ]:
appended_ids

In [ ]:
images_to_add

In [ ]:
pts2